In [1]:
from sklearn.model_selection import ShuffleSplit, cross_val_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.pipeline import Pipeline

from mne.decoding import CSP
from mne.channels import read_layout
from loaddata import *
from sklearn import metrics as met

print(__doc__)

Automatically created module for IPython interactive environment


In [2]:
fp = {
      'aa' : {
          'cnt': './data/BCI_III_Dataset_4/aa/100Hz/data_set_IVa_aa_cnt.txt',
          'mrk': './data/BCI_III_Dataset_4/aa/100Hz/data_set_IVa_aa_mrk.txt',
          'lab': './data/BCI_III_Dataset_4/aa/TRUE_LABELS.txt',
          'chn': './data/BCI_III_Dataset_4/aa/100Hz/ch_names.txt',
          'pos':168,
          'freq': 100
          },
       'al' : {
          'cnt': './data/BCI_III_Dataset_4/al/100Hz/data_set_IVa_al_cnt.txt',
          'mrk': './data/BCI_III_Dataset_4/al/100Hz/data_set_IVa_al_mrk.txt',
          'lab': './data/BCI_III_Dataset_4/al/TRUE_LABELS.txt',
          'chn': './data/BCI_III_Dataset_4/al/100Hz/ch_names.txt',
          'pos':224,
          'freq': 100
          },
       'av' : {
          'cnt': './data/BCI_III_Dataset_4/av/100Hz/data_set_IVa_av_cnt.txt',
          'mrk': './data/BCI_III_Dataset_4/av/100Hz/data_set_IVa_av_mrk.txt',
          'lab': './data/BCI_III_Dataset_4/av/TRUE_LABELS.txt',
          'chn': './data/BCI_III_Dataset_4/av/100Hz/ch_names.txt',
          'pos':84,
          'freq': 100
          },
       'aw' : {
          'cnt': './data/BCI_III_Dataset_4/aw/100Hz/data_set_IVa_aw_cnt.txt',
          'mrk': './data/BCI_III_Dataset_4/aw/100Hz/data_set_IVa_aw_mrk.txt',
          'lab': './data/BCI_III_Dataset_4/aw/TRUE_LABELS.txt',
          'chn': './data/BCI_III_Dataset_4/aw/100Hz/ch_names.txt',
          'pos':56,
          'freq': 100
          },
       'ay' : {
          'cnt': './data/BCI_III_Dataset_4/ay/100Hz/data_set_IVa_ay_cnt.txt',
          'mrk': './data/BCI_III_Dataset_4/ay/100Hz/data_set_IVa_ay_mrk.txt',
          'lab': './data/BCI_III_Dataset_4/ay/TRUE_LABELS.txt',
          'chn': './data/BCI_III_Dataset_4/ay/100Hz/ch_names.txt',
          'pos':28,
          'freq': 100
          }
}

#pick_chan = {
#    'al': ['C3', 'Cz', 'C5'],
#}

low_freq, high_freq = 7., 30.
tmin, tmax = 1., 2.

# event_id
event_id = {'right': 1, 'foot': 2}
    

In [3]:
def load(sujeto):
    #Se carga set de datos crudos
    raw = creatRawArray(sujeto)

    #Se carga eventos
    events, labels = creatEventsArray(sujeto)

    # Se aplica filtros band-pass
    raw.filter(low_freq, high_freq, fir_design='firwin', skip_by_annotation='edge')

    #Se genera las epocas con los datos crudos y los eventos
    epochs = mne.Epochs(raw, events=events, event_id=event_id, tmin=tmin, tmax=tmax, baseline=None, preload=True,
                    verbose=False)

    #Se carga labels (convierte 1 -> -1 y 2 -> 0 )
    labels = epochs.events[:, -1] - 2

    #Lo convierte a matriz numpy
    epochs_data = epochs.get_data()

    #Se crea set de de pruebas y test
    train_x=epochs_data[:sujeto['pos']]
    train_y=labels[:sujeto['pos']]

    test_x =epochs_data[sujeto['pos']:]
    test_y=labels[sujeto['pos']:]
    
    return train_x, train_y, test_x, test_y, epochs


In [4]:
for key, sujeto in fp.items():
    print(sujeto)
    train_x, train_y, test_x, test_y, epochs = load(sujeto)
    
    #Assemble a classifier
    lda = LinearDiscriminantAnalysis()
    csp = CSP(n_components=len(epochs.ch_names), reg=None, log=True, norm_trace=False)
    
    #Use scikit-learn Pipeline
    clf = Pipeline([('CSP', csp), ('LDA', lda)])
    
    clf.fit(train_x, train_y)
    result=clf.predict(test_x)
    
    print( met.confusion_matrix(test_y, result) )
    print( "Recall: ", met.recall_score(test_y, result, average=None) )
    print( "Precision: ", met.precision_score(test_y, result, average=None) )
    print( "F1: ", met.f1_score(test_y, result, average=None) )
    print( "Acurracy: ", met.accuracy_score(test_y, result) )
    
    tn, fp, fn, tp = met.confusion_matrix(test_y, result).ravel()
    print(tn, fp, fn, tp)
    
    print(met.classification_report(test_y, result))

{'cnt': './data/BCI_III_Dataset_4/aa/100Hz/data_set_IVa_aa_cnt.txt', 'mrk': './data/BCI_III_Dataset_4/aa/100Hz/data_set_IVa_aa_mrk.txt', 'lab': './data/BCI_III_Dataset_4/aa/TRUE_LABELS.txt', 'chn': './data/BCI_III_Dataset_4/aa/100Hz/ch_names.txt', 'pos': 168, 'freq': 100}
Creating RawArray with float64 data, n_channels=118, n_times=298458
    Range : 0 ... 298457 =      0.000 ...  2984.570 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 7 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 7.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 6.00 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 165 samples (1.650 sec)

Computing rank from data with ran

Creating RawArray with float64 data, n_channels=118, n_times=283562
    Range : 0 ... 283561 =      0.000 ...  2835.610 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 7 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 7.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 6.00 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 165 samples (1.650 sec)

Computing rank from data with rank=None
    Using tolerance 72 (2.2e-16 eps * 118 dim * 2.8e+15  max singular value)
    Estimated rank (mag): 118
    MAG: rank 118 computed from 118 data channels with 0 projectors
Reducing data rank from 118 -> 118
Estimating covariance using EMPIRICAL
Done.
Computing ra